In [37]:
#ignore the warnings
import warnings
warnings.filterwarnings("ignore")


from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Initilize the LLM
llm =ChatGroq(model ="llama-3.3-70b-versatile")

#Set the Cache
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
set_llm_cache(InMemoryCache())

In [ ]:
from typing import Literal, TypedDict
import uuid

from langgraph.constants import END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage

In [39]:
# Define the shared graph state
class State(TypedDict):
    message:str
    llm_output: str
    feedback: str

In [ ]:
# Simulate an LLM output node
def generate_llm_output(state: State) -> State:

    prompt = f"""
You are an expert WhatsApp assistant.  
Your task is to write a clear, polite, and professional WhatsApp reply.

Guidelines:
- Keep the response concise (under 30 words).  
- Maintain a friendly and professional tone.  
- If there is previous feedback, naturally incorporate it into your reply.  
- If the user says “Hi” or “How are you?”, reply casually and conversationally.  
- If the user asks about your availability or time, reply with: “I’m free from 11:30 PM to 12:10 AM.”  

User message:
{state["message"]}

Previous feedback:
{state["feedback"]}
"""
    response=llm.invoke([
        SystemMessage("You are the helpful AI whatsapp assistant"),
        HumanMessage(content=prompt),
        
    ])
    result=response.content
    return {"llm_output": result}


In [41]:

# Human approval node
def human_approval(state: State) -> Command[Literal["approved_path", "rejected_path"]]:
    decision = interrupt({
        "question": "Do you approve the following output?",
        "llm_output": state["llm_output"]
    })

    if decision == "yes":
        return Command(goto="approved_path", update={"feedback": "approved"})
    else:
        return Command(goto="rejected_path", update={"feedback": "rejected"})

In [ ]:
# Next steps after approval
def approved_node(state: State) -> State:

    print("✅ Approved path taken.")
    return {"llm_output":state["llm_output"]}

# Alternative path after rejection
def rejected_node(state: State) -> State:
    print("❌ Rejected path taken.")
    

In [43]:
# Build the graph
builder = StateGraph(State)
builder.add_node("generate_llm_output", generate_llm_output)
builder.add_node("human_approval", human_approval)
builder.add_node("approved_path", approved_node)
builder.add_node("rejected_path", rejected_node)

builder.set_entry_point("generate_llm_output")
builder.add_edge("generate_llm_output", "human_approval")
builder.add_edge("approved_path", END)
builder.add_edge("rejected_path", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [ ]:
graph.get_graph().print_ascii()

                +-----------+                  
                | __start__ |                  
                +-----------+                  
                      *                        
                      *                        
                      *                        
           +---------------------+             
           | generate_llm_output |             
           +---------------------+             
                      *                        
                      *                        
                      *                        
              +----------------+               
              | human_approval |               
              +----------------+               
               ..            ..                
             ..                ..              
           ..                    ..            
+---------------+           +---------------+  
| approved_path |           | rejected_path |  
+---------------+           +-----------

In [44]:

# Run until interrupt
config = {"configurable": {"thread_id": uuid.uuid4()}}
query=input("Write the message")
result = graph.invoke({"message":query,
                       "feedback":""
                       }, config=config)
print(result["__interrupt__"])

ask=input("Approve or Reject")
final_result = graph.invoke(Command(resume=ask), config=config)
print(final_result)

[Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': 'I’m free from 11:30 PM to 12:10 AM.'}, id='2cad939fb3c664e9a8652926eb48de60')]
✅ Approved path taken.
{'message': 'are you free amir?', 'llm_output': 'I’m free from 11:30 PM to 12:10 AM.', 'feedback': 'approved'}


In [46]:

# Run until interrupt
config = {"configurable": {"thread_id": uuid.uuid4()}}
query=input("Write the message")
result = graph.invoke({"message":query,
                       "feedback":""
                       }, config=config)
print(result["__interrupt__"])

ask=input("Approve or Reject")
final_result = graph.invoke(Command(resume=ask), config=config)
print(final_result)

[Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': "I'm doing great, thanks for asking!"}, id='abe8812d74e0bce0d23a9c6004923b13')]
✅ Approved path taken.
{'message': 'how are you amir?', 'llm_output': "I'm doing great, thanks for asking!", 'feedback': 'approved'}
